In [76]:
from microplate import PACKAGE_PATH
import os 
from airo_dataset_tools.data_parsers.coco import CocoKeypointAnnotation, CocoKeypointsDataset,CocoKeypointCategory,CocoImage
import cv2
import numpy as np
import json

In [77]:
data_dir = PACKAGE_PATH.parents[1] / 'data'
scenes = os.listdir(data_dir)
images = []
for scene in scenes:
    scene_images = (os.listdir(data_dir / scene))
    images.extend([scene + '/' + image for image in scene_images])
images = [ image for image in images if image.endswith('.png') ]

In [78]:
print(f"found {len(images)} images")

found 175 images


In [79]:
coco_category = CocoKeypointCategory(supercategory='microplate',id=1, name='microplate',keypoints=['topleft','topright','bottomleft'])
coco_images = []
coco_annotations = []
for image_idx,image_path in enumerate(images):
    image = cv2.imread(str(data_dir/image_path))
    height, width = image.shape[:2]
    image_id = int(image_idx)
    coco_image = CocoImage( file_name=image_path, height=height, width=width, id=image_id)
    coco_images.append(coco_image)
    annotations_path = image_path.replace('.png','_keypoints.json')
    annotations_path = data_dir/annotations_path
    with open(annotations_path,"r") as f:
        annotations = json.load(f)
    # ! asume the order matches the category order and all are present in the dict.
    keypoints = []
    num_keypoints = 0
    assert len(annotations) == 3, f"Expected 3 keypoints, got {len(annotations)}, {image_path}"
    for category,keypoint in annotations.items():
        keypoints.extend(keypoint)
        if 0 <= keypoint[0] <= width and 0 <= keypoint[1] <= height:
            visibility = 2
            num_keypoints += 1
        else:
            visibility = 0
        keypoints.extend([visibility])
    coco_annotation = CocoKeypointAnnotation(id=image_id,image_id=image_id, category_id=1, keypoints=keypoints, num_keypoints=num_keypoints)
    coco_annotations.append(coco_annotation)  


In [80]:
coco_images[0]

CocoImage(id=0, width=1280, height=720, file_name='2/20230803_183218.png', license=None, flicker_url=None, coco_url=None, date_captured=None)

In [81]:
coco_annotations[0]

CocoKeypointAnnotation(id=0, image_id=0, category_id=1, bbox=None, segmentation=None, area=None, iscrowd=None, keypoints=[623.0, 171.0, 2.0, 807.0, 391.0, 2.0, 453.0, 267.0, 2.0], num_keypoints=3)

In [82]:
coco_dataset = CocoKeypointsDataset(images=coco_images, annotations=coco_annotations, categories=[coco_category])

In [87]:
with open(data_dir / 'annotations.json',"w") as f:
    json.dump(coco_dataset.dict(), f)